In [3]:
import yfinance as yf
import numpy as np
import pandas as pd
import datetime as dt
from scipy.stats import norm

# Define the stock symbols
symbols = ['AAPL', 'AMZN', 'NVDA', 'TSLA', 'ANET']

# Define trading parameters
initial_capital = 10000  # Initial capital in USD
risk_per_trade = 0.02  # Percentage of capital to risk per trade
stop_loss_pct = 0.05  # Maximum percentage loss per trade
target_profit_pct = 0.1  # Target percentage profit per trade
num_simulations = 10000  # Number of Monte Carlo simulations

# Function to calculate historical volatility
def calculate_historical_volatility(data):
    log_returns = np.log(data['Close'] / data['Close'].shift(1))
    historical_volatility = np.sqrt(252) * log_returns.std()  # 252 trading days in a year
    return historical_volatility

# Function to calculate option price using Monte Carlo simulation
def monte_carlo_option_price(S, K, T, r, sigma, num_simulations):
    z = np.random.standard_normal(num_simulations)
    ST = S * np.exp((r - 0.5 * sigma ** 2) * T + sigma * np.sqrt(T) * z)
    payoff = np.maximum(ST - K, 0)
    option_price = np.exp(-r * T) * np.mean(payoff)
    return option_price

# Retrieve historical market data and calculate historical volatility
historical_data = {}
for symbol in symbols:
    historical_data[symbol] = yf.download(symbol, start="2022-01-01", end="2024-02-29")
    historical_data[symbol]['HistoricalVolatility'] = calculate_historical_volatility(historical_data[symbol])

# Define trading strategy
def simple_ma_crossover_strategy(data):
    # Moving averages
    data['MA50'] = data['Close'].rolling(window=50).mean()
    data['MA200'] = data['Close'].rolling(window=200).mean()

    # Entry/Exit signals
    data['Signal'] = 0
    data.iloc[50:, data.columns.get_loc('Signal')] = np.where(data['MA50'][50:] > data['MA200'][50:], 1, 0)
    data['Position'] = data['Signal'].diff()

    return data

if __name__ == "__main__":
    # Prompt user to choose symbol
    chosen_symbol = input("Enter the symbol you want to analyze (e.g., AAPL, AMZN, NVDA, TSLA, ANET): ")

    if chosen_symbol not in symbols:
        print("Invalid symbol. Please choose from the list of symbols: AAPL, AMZN, NVDA, TSLA, ANET")
    else:
        # Backtest trading strategy for the chosen symbol
        data = historical_data[chosen_symbol]
        data = simple_ma_crossover_strategy(data)

        # Split data into training and testing sets
        train_data = data[data.index < '2022-11-01']
        test_data = data[data.index >= '2022-11-01']

        # Initialize positions and capital
        positions = pd.DataFrame(index=train_data.index, columns=['Position'])
        positions = positions.fillna(0)
        capital = initial_capital

        # Simulate trades using training data
        for i in range(len(train_data)):
            if train_data['Position'][i] == 1:
                # Buy signal
                if positions['Position'][i-1] == 0:
                    risk = capital * risk_per_trade
                    option_price = train_data['Close'][i]  # Assuming option price is equal to stock price
                    volatility = train_data['HistoricalVolatility'][i]
                    time_to_expiry = (dt.datetime.strptime("2022-12-31", "%Y-%m-%d") - train_data.index[i]).days / 252  # Time to expiry in years
                    monte_carlo_price = monte_carlo_option_price(option_price, option_price, time_to_expiry, 0, volatility, num_simulations)
                    contracts_to_buy = int(risk / (monte_carlo_price * (1 + stop_loss_pct)))
                    capital -= contracts_to_buy * monte_carlo_price
                    positions['Position'][i] = contracts_to_buy

            elif train_data['Position'][i] == -1:
                # Sell signal
                if positions['Position'][i-1] > 0:
                    option_price = train_data['Close'][i]  # Assuming option price is equal to stock price
                    capital += positions['Position'][i-1] * option_price
                    positions['Position'][i] = 0

        # Calculate final capital and performance metrics using testing data
        final_value = capital + positions['Position'][-1] * test_data['Close'][-1]
        pnl = final_value - initial_capital
        print(f"Symbol: {chosen_symbol}, Final Value: {final_value}, P&L: {pnl}")

        

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Enter the symbol you want to analyze (e.g., AAPL, AMZN, NVDA, TSLA, ANET): AAPL
Symbol: AAPL, Final Value: 10000.0, P&L: 0.0
